# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [41]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [42]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,0,bubaque,GW,2024-05-09,11.2833,-15.8333,100.0,81.0,79.934,3.31
1,1,port-aux-francais,TF,2024-05-09,-49.3500,70.2167,100.0,91.0,44.330,16.89
2,2,antalaha,MG,2024-05-09,-14.9003,50.2788,79.0,88.0,73.580,3.29
3,3,howard springs,AU,2024-05-09,-12.4970,131.0470,5.0,66.0,92.084,4.70
4,4,saint-pierre,RE,2024-05-09,-21.3393,55.4781,75.0,83.0,73.346,7.72


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [43]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df["Humidity (%)"].astype(float)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity (%)",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity (%))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [44]:
# Narrow down cities that fit criteria and drop any results with null values

#A max temperature lower than 75 degrees but higher than 10
city_data_df = city_data_df.loc[(city_data_df["Max Temp (F)"] < 70) & (city_data_df["Max Temp (F)"] > 10)]

#Wind speed less than 5 mph
city_data_df = city_data_df.loc[(city_data_df["Wind Speed (mph)"] < 5)]

#Cloudiness less than 25%
city_data_df = city_data_df.loc[(city_data_df["Cloudiness (%)"] < 25)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
14,14,olbia,IT,2024-05-09,40.9214,9.4856,0.0,72.0,62.924,1.03
16,16,bredasdorp,ZA,2024-05-09,-34.5322,20.0403,0.0,82.0,50.648,2.32
17,17,waiuku,NZ,2024-05-09,-37.2500,174.7500,0.0,70.0,60.224,4.85
39,39,la sarre,CA,2024-05-09,48.8002,-79.1996,8.0,58.0,43.700,2.97
42,42,mukharram al fawqani,SY,2024-05-09,34.8151,37.0861,2.0,28.0,65.354,4.70


### Step 3: Create a new DataFrame called `hotel_df`.

In [53]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = pd.DataFrame({"City": city_data_df["City"],
                        "Country": city_data_df["Country"],
                        "Latitude": city_data_df["Latitude"],
                        "Longitude": city_data_df["Longitude"],
                        "Humidity (%)": city_data_df["Humidity (%)"]})

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Latitude,Longitude,Humidity (%),Hotel Name
14,olbia,IT,40.9214,9.4856,72.0,
16,bredasdorp,ZA,-34.5322,20.0403,82.0,
17,waiuku,NZ,-37.2500,174.7500,70.0,
39,la sarre,CA,48.8002,-79.1996,58.0,
42,mukharram al fawqani,SY,34.8151,37.0861,28.0,
...,...,...,...,...,...,...
519,nantucket,US,41.2835,-70.0995,93.0,
545,brejo da madre de deus,BR,-8.1458,-36.3711,92.0,
556,ulysses,US,37.5814,-101.3552,58.0,
561,egilsstadir,IS,65.2653,-14.3948,87.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [54]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Latitude"]
    lon = row["Longitude"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
olbia - nearest hotel: City Garden Guest House
bredasdorp - nearest hotel: Victoria Hotel
waiuku - nearest hotel: The Kentish Hotel
la sarre - nearest hotel: Motel le Bivouac
mukharram al fawqani - nearest hotel: No hotel found
talnakh - nearest hotel: Талнах
jose maria morelos - nearest hotel: No hotel found
pacific grove - nearest hotel: Pacific Grove Inn
ruidoso - nearest hotel: Hotel Ruidoso
viedma - nearest hotel: Níjar
lebu - nearest hotel: No hotel found
thunder bay - nearest hotel: Hampton Inn & Suites by Hilton Thunder Bay
metlili chaamba - nearest hotel: No hotel found
stanley - nearest hotel: Hotel 52
port alfred - nearest hotel: No hotel found
mao - nearest hotel: Hevresac
sao francisco - nearest hotel: Hotel Atalaia
marquette - nearest hotel: No hotel found
illapel - nearest hotel: Domingo Ortiz de Rosas
port lincoln - nearest hotel: Boston Hotel
akureyri - nearest hotel: Hótel Norðurland
altay - nearest hotel: Sunshine Holiday Hotel
ribeira brava - n

,City,Country,Latitude,Longitude,Humidity (%),Hotel Name
14,olbia,IT,40.9214,9.4856,72.0,City Garden Guest House
16,bredasdorp,ZA,-34.5322,20.0403,82.0,Victoria Hotel
17,waiuku,NZ,-37.2500,174.7500,70.0,The Kentish Hotel
39,la sarre,CA,48.8002,-79.1996,58.0,Motel le Bivouac
42,mukharram al fawqani,SY,34.8151,37.0861,28.0,No hotel found
...,...,...,...,...,...,...
519,nantucket,US,41.2835,-70.0995,93.0,No hotel found
545,brejo da madre de deus,BR,-8.1458,-36.3711,92.0,No hotel found
556,ulysses,US,37.5814,-101.3552,58.0,Single Tree Inn Hotel O
561,egilsstadir,IS,65.2653,-14.3948,87.0,Valaskjálf


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [55]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Hotel Name,Country)